<a href="https://colab.research.google.com/github/E23A48/TF_MachineLearning/blob/main/TF_MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TF**
**MACHINE LEARNING**
2020-2

Integrantes:
*   Piero Herrera
*   Camilo Silva
*   Diego Urrutia

#**Introducción :**

#**Convolutional Neural Network**

In [61]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

import cv2
import matplotlib.pyplot as plt


In [62]:
train_path = 'data/train'
training_names = os.listdir(train_path)


image_paths = []
#Etiquetas
image_classes = []

class_id = 0


def imglist(path):
    return [os.path.join(path,f) for f in os.listdir(path)]


for training_name in training_names:
    dir = os.path.join(train_path, training_name)
    class_path = imglist(dir)
    image_paths += class_path
    image_classes += [class_id]*len(class_path)
    class_id += 1


for image_path in image_paths:
    im = cv2.imread(image_path)



y = np.array(image_classes)


In [63]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 1)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()




Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 50, 50, 32)        896       
_________________________________________________________________
activation_28 (Activation)   (None, 50, 50, 32)        0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 8, 8, 32)          9248      
_________________________________________________________________
activation_29 (Activation)   (None, 8, 8, 32)          0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 1, 1, 64)         

In [64]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
